# FCC Calorimeter Performance Studies Workflow

## Overview

This tutorial will teach you how to:

* **simulate** the single particle response of the calorimeter detector system
* **reconstruct** physics object from raw signals
* produce **plots** of energy resolutions and other quantities.

First, make sure your setup of the FCC software is working. 
You can check that the command to run jobs in the Gaudi framework is available on the command line:

In [1]:
!which fccrun

/usr/local/scripts//fccrun


If you don't see a valid path like `/usr/local/bin/fccrun`  you should consult [the documentation page on FCCSW setup](https://github.com/vvolkl/fcc-tutorials/blob/master/FccSoftwareGettingStarted.md)

## Using the DD4hep detector model in FCC Software.

The Geant4 geometry is used for the full simulation of the detector is not written directly, but generated using the DD4hep library.
The detector description in this library consists of two parts:
A compiled C++ library that constructs the geometry, and a set of xml files that contain parameters that can are parsed by the library at runtime and make the detector geometry (somewhat) configurable.
In the framework, the geometry is available to all components via the service `GeoSvc`.
One of the simplest jobs is to write the geometry to a `.gdml` file:

In [9]:
%%writefile dumpGeo_fccee.py

import os
from Gaudi.Configuration import *
from Configurables import GeoSvc
geoservice = GeoSvc("GeoSvc")
geoservice.detectors=[
	os.path.join(os.environ.get("FCC_DETECTORS", ""), 'Detector/DetFCCeeIDEA/compact/FCCee_DectEmptyMaster.xml'),
  os.path.join(os.environ.get("FCC_DETECTORS", ""), 'Detector/DetFCCeeIDEA/compact/FCCee_DectMaster.xml'),
                                         ]

from Configurables import SimG4Svc
geantservice = SimG4Svc("SimG4Svc")

from Configurables import GeoToGdmlDumpSvc
geodumpservice = GeoToGdmlDumpSvc("GeoDump") 
geodumpservice.gdml="FCCee_IDEA.gdml"

from Configurables import ApplicationMgr
ApplicationMgr( TopAlg = [], 
                EvtSel = 'NONE',
                EvtMax   = 1,
                # order is important, as GeoSvc is needed by SimG4Svc
                ExtSvc = [geoservice, geantservice, geodumpservice],
                OutputLevel=DEBUG
 )


Overwriting dumpGeo_fccee.py


A job with this configuration can be executed with 

In [11]:
!FCC_DETECTORS=/usr/local fccrun dumpGeo_fccee.py

 

JobOptionsSvc       DEBUG Service base class initialized successfully
MessageSvc          DEBUG Service base class initialized successfully
ApplicationMgr      DEBUG Getting my own properties
ApplicationMgr    SUCCESS 
                                                   Welcome to ApplicationMgr (GaudiCoreSvc v32r0)
                                          running on vali-XPS-13-9360 on Mon Sep  9 18:10:21 2019
ApplicationMgr      DEBUG Loading declared DLL's

**************************************************************
 Geant4 version Name: geant4-10-04-patch-02 [MT]   (25-May-2018)
                       Copyright : Geant4 Collaboration
                      References : NIM A 506 (2003), 250-303
                                 : IEEE-TNS 53 (2006), 270-278
                                 : NIM A 835 (2016), 186-225
                             WWW : http://geant4.org/
**************************************************************

ApplicationMgr       INFO Application Manager

RndmGenSvc          DEBUG Service base class initialized successfully
RndmGenSvc.Engine   DEBUG Service base class initialized successfully
RndmGenSvc.Engine    INFO Generator engine type:CLHEP::RanluxEngine
RndmGenSvc.Engine    INFO Current Seed:1234567 Luxury:3
RndmGenSvc           INFO Using Random engine:HepRndm::Engine<CLHEP::RanluxEngine>
EventDataSvc        DEBUG Service base class initialized successfully
EventPersistenc...  DEBUG Service base class initialized successfully
SimG4Svc.SimG4D...  DEBUG Initialize base class GaudiCommon<CounterHolder<AlgTool>>
SimG4Svc.SimG4D...  DEBUG could not locate CounterSummarySvc, no counter summary will be made
SimG4Svc.SimG4D...  DEBUG List of ALL properties of SimG4DD4hepDetector/SimG4Svc.SimG4DD4hepDetector  #properties = 26
SimG4Svc.SimG4D...  DEBUG Property ['Name': Value] =  'GeoSvc':GeoSvc
SimG4Svc.SimG4D...  DEBUG Property ['Name': Value] =  'ContextService':AlgContextSvc
SimG4Svc.SimG4D...  DEBUG Property ['Name': Value] =  'StatEn

         --->  Isotope:   O17   Z =  8   N =  17   A =  17.00 g/mole   abundance:  0.038 %
         --->  Isotope:   O18   Z =  8   N =  18   A =  18.00 g/mole   abundance:  0.205 %
          ElmMassFraction:  20.92 %  ElmAbundance  12.82 % 

Geant4Converter  DEBUG ++ Volume     + LumiCal_layer0slice1 converted: 0x558ba4c2fbc0 ---> G4: 0x558ba55bb400
Geant4Converter  DEBUG ++ Convert Volume LumiCal_layer0slice2            : 0x558ba4c30090 TGeoTubeSeg/TGeoVolume assembly:NO 
Geant4Converter  DEBUG ++ Created G4 Isotope SI from data: Z=14 N=28 A=28.085 [g/mole]
Geant4Converter  DEBUG ++ Created G4  Element: SILICON (SI)   Z = 14.0   N =    28   A = 28.085 g/mole
         --->  Isotope:  SI28   Z = 14   N =  28   A =  27.98 g/mole   abundance: 92.230 %
         --->  Isotope:  SI29   Z = 14   N =  29   A =  28.98 g/mole   abundance:  4.683 %
         --->  Isotope:  SI30   Z = 14   N =  30   A =  29.97 g/mole   abundance:  3.087 % No.Isotopes:0
Geant4Converter  DEBUG ++ Created G4  Materi


 FTFP_BERT : new threshold between BERT and FTFP is over the interval 
 for pions :   3 to 12 GeV
 for kaons :   3 to 12 GeV
 for proton :  3 to 12 GeV
 for neutron : 3 to 12 GeV

### Adding tracking cuts for neutron  TimeCut(ns)= 10000  KinEnergyCut(MeV)= 0
even though it appears in the current tracking world.
Default cuts are used for this region.
even though it appears in the current tracking world.
Default cuts are used for this region.
SimG4Svc             INFO Random numbers seeds: 1234567	139989490523296

phot:   for  gamma    SubType= 12  BuildTable= 0
      LambdaPrime table from 200 keV to 100 TeV in 61 bins 
      ===== EM models for the G4Region  DefaultRegionForTheWorld ======
 LivermorePhElectric :  Emin=        0 eV    Emax=      100 TeV   AngularGenSauterGavrila  FluoActive

compt:   for  gamma    SubType= 13  BuildTable= 1
      Lambda table from 100 eV  to 1 MeV, 7 bins per decade, spline: 1
      LambdaPrime table from 1 MeV to 100 TeV in 56 bins 
      ===== EM mod

        WentzelVIUni :  Emin=        0 eV    Emax=      100 TeV  Table with 84 bins Emin=    100 eV    Emax=    100 TeV

hIoni:   for  anti_proton    SubType= 2
      dE/dx and range tables from 100 eV  to 100 TeV in 84 bins
      Lambda tables from threshold to 100 TeV, 7 bins per decade, spline: 1
      finalRange(mm)= 0.1, dRoverRange= 0.2, integral: 1, fluct: 1, linLossLimit= 0.01
      ===== EM models for the G4Region  DefaultRegionForTheWorld ======
            ICRU73QO :  Emin=        0 eV    Emax=        2 MeV
          BetheBloch :  Emin=        2 MeV   Emax=      100 TeV

hBrems:   for  anti_proton    SubType= 3
      dE/dx and range tables from 100 eV  to 100 TeV in 84 bins
      Lambda tables from threshold to 100 TeV, 7 bins per decade, spline: 1
      ===== EM models for the G4Region  DefaultRegionForTheWorld ======
               hBrem :  Emin=        0 eV    Emax=      100 TeV

hPairProd:   for  anti_proton    SubType= 4
      dE/dx and range tables from 100 eV  to 100 

      dE/dx and range tables from 100 eV  to 100 TeV in 84 bins
      Lambda tables from threshold to 100 TeV, 7 bins per decade, spline: 1
      Sampling table 21x1001; from 1 GeV to 100 TeV 
      ===== EM models for the G4Region  DefaultRegionForTheWorld ======
          muPairProd :  Emin=        0 eV    Emax=      100 TeV

CoulombScat:   for  mu-, integral: 1     SubType= 1  BuildTable= 1
      Used Lambda table of mu+
      ThetaMin(p) < Theta(degree) < 180; pLimit(GeV^1)= 0.139531
      ===== EM models for the G4Region  DefaultRegionForTheWorld ======
  eCoulombScattering :  Emin=        0 eV    Emax=      100 TeV

msc:   for pi+    SubType= 10
      RangeFactor= 0.2, stepLimitType: 0, latDisplacement: 0
      ===== EM models for the G4Region  DefaultRegionForTheWorld ======
        WentzelVIUni :  Emin=        0 eV    Emax=      100 TeV  Table with 84 bins Emin=    100 eV    Emax=    100 TeV

hIoni:   for  pi+    SubType= 2
      dE/dx and range tables from 100 eV  to 100 TeV i

ServiceManager      DEBUG Initializing service GeoDump
GeoDump             DEBUG Service base class initialized successfully
======       Pre-compound/De-excitation Physics Parameters     ========
Type of pre-compound inverse x-section              3
Pre-compound model active                           1
Pre-compound low energy (MeV)                       0.1
Type of de-excitation inverse x-section             3
Type of de-excitation factory                       Evaporation
Number of de-excitation channels                    8
Min excitation energy (keV)                         0.01
Min energy per nucleon for multifragmentation (MeV) 1e+05
Level density (1/MeV)                               0.1
Time limit for long lived isomeres (ns)             1e+12
Internal e- conversion flag                         1
Store e- internal conversion data                   0
Electron internal conversion ID                     2
Correlated gamma emission flag                      0
Max 2J for sampling of

Note the printout of the GeoSvc and make sure the information is as expected. If there is something unclear or missing make sure to open an [issue](https://github.com/HEP-FCC/FCCSW/issues)!
Take a look at the resulting gdml file. Although it is text-based it is not really human-readable for a geometry of this size, but you can check the number of lines and volume names if you are familiar with the geometry.

In [15]:
!tail FCCee_IDEA.gdml

        <volumeref ref="Vertex0x558ba55e92d0"/>
      </physvol>
    </volume>
  </structure>

  <setup name="Default" version="1.0">
    <world ref="world_volume0x558ba55e94b0"/>
  </setup>

</gdml>


## Running Geant4 within the FCC Software Framework

To run Geant4, a number of additional components are required, foremost the `SimG4Alg` and `SimG4Svc`.
The simplest way to generate a particle to simulate is to use one of the particle guns (`GenAlg` with the `MomentumRangeParticleGun` tool for example).
This algorithm produces a particles with energies from a flat probability distribution in HepMC format.
To use them as input they first need to be converted to the FCC event data model (`HepMCToEDMConverter`) and piped into the Geant4 interface via the `SimG4PrimariesFromEdmTool`.

A configuration that runs all of this is distributed with FCCSW


The output of this job is `fccee_idea_LAr_pgun.root`, a ROOT file containing the simulation products of 500 single particle events (5 Gev e-) in the FCC event data model.
Check for example that the distribution of the input particles is as you expect:

In [ ]:
import ROOT
f = ROOT.TFile("fccee_idea_LAr_pgun.root")
events = f.Get("events")
c = ROOT.TCanvas()
events.Draw("sqrt(pow(GenParticles.core.p4.px,2) + pow(GenParticles.core.p4.py,2) +pow(GenParticles.core.p4.pz,2))")



The exact position of the energy deposit is available too:

In [ ]:

import ROOT
f = ROOT.TFile("fccee_idea_LAr_pgun.root")
events = f.Get("events")
c = ROOT.TCanvas()
events.Draw("ECalBarrelPositions.position.y:ECalBarrelPositions.position.x", "", "", 5, 0)

### Obtaining and Plotting the Energy Resolution

Now that the detector response is simulated, it is time to reconstruct the signals. FCCSW includes another configuration to run a Sliding Window reconstruction:

In [ ]:
!fccrun $FCCSWBASEDIR/Reconstruction/RecFCCeeCalorimeter/options/runFullCaloSystem_ReconstructionSW_noiseFromFile.py --inputs fcc_idea_LAr_pgun.root -n 500

This configuration inludes electronics noise especially calculated for this detector geometry.

TODO: Plotting scripts

### Further Topics: Calculating the Sampling Fraction and using it in Simulation

TODO








